In [ ]:
%pylab inline
import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd

import sys
from importlib import reload

sys.path.append('../../code/scripts')
import utils
reload(utils)
from utils import read_subset_results_nonimage, get_best_hp_results
from dataset_params import dataset_params

import plotting as p


In [ ]:
#results_tag = 'no_demographics'
results_tag = 'with_demographics'
results_descriptor_ERM = 'mooc_subsetting_{0}_ERM'.format(results_tag)
results_general_path = '../../results/subset_results/'
this_results_path_ERM = os.path.join(results_general_path, results_descriptor_ERM)

group_key = 'post_secondary'

#plotting
scale = 5
acc_key = '1 - auc_roc'

groups = [0,1]
group_pair = [r'edu $\leq$ secondary',r'edu $>$ secondary']
group_id_dict = {0: group_pair[0], 1: group_pair[1]}

gamma0 = dataset_params['mooc']['gamma']
gammas = [gamma0,1-gamma0]

# 1. read in results

In [ ]:
reload(utils)

pred_fxn_name = 'rf_classifier'
if pred_fxn_name == 'rf_classifier':
    if results_tag == 'with_demographics':
        max_depth = 16
    else:
        max_depth = 8
    param_dict = {'max_depth': [max_depth], 'n_estimators': [400]}
    num_seeds_eval = 10

obj_str = 'ERM'
results_descriptor = results_descriptor_ERM

pred_fxn_base_name = 'subset_{0}'.format(group_key, pred_fxn_name)

this_results_path = os.path.join(results_general_path, results_descriptor)
results_path_this_pred_fxn = os.path.join(this_results_path,pred_fxn_base_name, pred_fxn_name)
 
these_keys = tuple([x[0] for x in param_dict.values()])
print(results_path_this_pred_fxn)
r_ERM = utils.read_subset_results_nonimage(results_path_this_pred_fxn,
                                   param_dict,
                                   by_seed=True,
                                   num_seeds = num_seeds_eval,
                                   acc_keys = ['acc','auc_roc'])[these_keys]

obj_str = 'IW'
results_descriptor = results_descriptor_ERM.replace('ERM', 'IW')

pred_fxn_base_name = 'subset_{0}'.format(group_key, pred_fxn_name)

this_results_path = os.path.join(results_general_path, results_descriptor)
results_path_this_pred_fxn = os.path.join(this_results_path,pred_fxn_base_name, pred_fxn_name)
 
#these_keys = tuple([x[0] for x in param_dict.values()])
print(results_path_this_pred_fxn)
r_IW = utils.read_subset_results_nonimage(results_path_this_pred_fxn,
                                   param_dict,
                                   #alternative_to_param_specifier='subset',
                                   by_seed=True,
                                   num_seeds = num_seeds_eval,
                                   acc_keys = ['acc','auc_roc'])[these_keys]

# 2. plot results

In [ ]:
# preload font sizes, etc.
reload(p)
fig, ax = p.setup_uplot_ax()

ls = ['-','--', ':']
obj_names = ['ERM', 'IW'] 
acc_key_plot = acc_key

results_to_plot = [r_ERM, r_IW]

if results_tag == 'with_demographics':
    gamma_annot_offset = (0.01,.021)
    
else:
    gamma_annot_offset = (0.01,0.027)

for i, results in enumerate(results_to_plot):
    subset_sizes = results['subset_sizes']    
    accs_by_group = results['accs_by_group']
    accs_total = results['accs_total']
    subset_fracs = subset_sizes / subset_sizes.sum(axis=0)
    accs_total_avgs = {}
    
    acc_key_replace = '1-aucroc'

    p.plot_by_group({acc_key_replace: accs_by_group[acc_key_plot]},
                    {acc_key_replace: accs_total[acc_key_plot]},
                    subset_fracs,
                    acc_key_replace, 
                    group_id_dict,
                    gammas = gammas,
                    pop_weights = 'by_eval_set',
                    ls = ls[i],
                    label_append = obj_names[i],
                    range_type='stddev',
                    title='Mooc certification prediction',
                    ylim=None,
                    plot_alpha_star=True, 
                    plot_gamma = True,
                    gamma_annot_offset = gamma_annot_offset,
                    ax = ax)
    
#ax.set_yscale('log')#loglog()
ax.grid(linewidth=1);
#ax.set_xlim(-0.01,1.01);
# if acc_key == 'acc':
#     ax.set_ylim(.525,.62);
if acc_key == '1 - auc_roc':
    ax.set_ylabel('1 - AUROC')
    ax.set_ylim(.02,.055);

plt.savefig('../../figures/uplot_mooc_{0}_auc.pdf'.format(results_tag), bbox_inches='tight')

In [ ]:
# preload font sizes, etc.
reload(p)
fig, ax = p.setup_uplot_ax_smaller()

ls = ['-','--', ':']
obj_names = ['ERM', 'IW'] 
acc_key_plot = acc_key

results_to_plot = [r_ERM, r_IW]

if results_tag == 'with_demographics':
    gamma_annot_offset = (0.01,.021)
    
else:
    gamma_annot_offset = (0.01,0.027)

for i, results in enumerate(results_to_plot):
    subset_sizes = results['subset_sizes']    
    accs_by_group = results['accs_by_group']
    accs_total = results['accs_total']
    subset_fracs = subset_sizes / subset_sizes.sum(axis=0)
    accs_total_avgs = {}
    
    acc_key_replace = '1-aucroc'

    p.plot_by_group({acc_key_replace: accs_by_group[acc_key_plot]},
                    {acc_key_replace: accs_total[acc_key_plot]},
                    subset_fracs,
                    acc_key_replace, 
                    group_id_dict,
                    gammas = gammas,
                    pop_weights = 'by_eval_set',
                    ls = ls[i],
                    label_append = obj_names[i],
                    range_type='stddev',
                    title='Mooc certification prediction',
                    ylim=None,
                    plot_alpha_star=True, 
                    plot_gamma = True,
                    group_labels_only=True,
                    gamma_annot_offset = gamma_annot_offset,
                    ax = ax)
    
#ax.set_yscale('log')#loglog()
ax.grid(linewidth=1);
#ax.set_xlim(-0.01,1.01);
# if acc_key == 'acc':
#     ax.set_ylim(.525,.62);
if acc_key == '1 - auc_roc':
    ax.set_ylabel('1 - AUROC')
    ax.set_ylim(.02,.055);

plt.savefig('../../figures/uplot_mooc_{0}_auc_wide.pdf'.format(results_tag), bbox_inches='tight')